In [107]:
# Yo! Pour expliquer un peu, j'ai codé de telle manière que les listes d'éléments, de nodes et de connectivité soient générés 
# automatiquement pour des treillis en triangle (ou en V? J'ai plus le nom), comme celui de l'exo 2.
# C'est pas forcément l'idéal niveau clarté, donc si tu préfères un autre fonctionnement (genre laisser l'utilisateur définir lui même
# les numéros de nodes, éléments et la connectivité), on peut faire comme ça aussi.

# Donc du coup, pour le code dessous, ça laisse pas de choix pour la définition du numéro de node. En gros, la première node à gauche (celle de l'appui)
# est la numéro 0, puis les numéro de nodes augmentent en zig zag en suivant les éléments inclinés du treillis.
# -> Node à gauche en bas, puis node à gauche en haut, puis node à droite de la première, puis node à droite de la deuxième...
# Pareil pour les éléments, le numéro 0 est celui tout à gauche, puis le 1 est l'horizontal partant de la node 0,
# puis le n°2 est celui reliant les nodes 1 et 2, puis le n°3 est celui reliant les nodes 2 et 3, et ainsi de suite pour chaque triangle
# du treillis.
# J'espère que ça aide, sinon on en discute quand on se voie!


import numpy as np

#For Δ trusses, define angle of first element (left bar of the first Δ, relative to the horizontal).
#Define number of Δ in the truss, will define number of elements automatically
#Assumptions:
# -Fixed supports on first and last nodes
# -a = length of v assemblage/2
# -b = heigth of the truss
# -Horizontal elements at top and bottom of truss
# First node starts at the left support, then follows the inclined elements to go to the next node. If defined well, last node correspond to the right support.


# Input data and element stiffness matrices
Einput = 200e3 #int(input("Elements E, Mpa:")) # N/mm^2
Ainput = 10e3 #int(input("Elements A, mm^2:")) #mm^2
a = 3000 #mm
b = 4000 #mm
P = 150e3 #N
phiP = -90 #°
ntriangle = 3 #int(input("Number of triangles in the truss:")) #Number of triangles shapes in the truss
nnodes_P = 3 #int(input("Number of nodes with load P applied:")) #Number of nodes with a load applied
added_node = [2,4,6]#[] #Nodes with load indexs
# for i in range(nnodes_P):
#     added_node.append(int(input(f"Node with load applied number {i+1}:")))

#Derived data from input and initialisation

nnodes = 2*ntriangle + 1
nelements = 3*ntriangle + (ntriangle - 1)
nodes_P = np.array(added_node)

E = np.zeros((nelements))+Einput  # N/mm^2
A = np.zeros((nelements))+Ainput #mm^2
theta1 = np.degrees(np.arctan(b/a)) #°
theta = [] #°

# Unit stiffness matrix of bar in local coordinate system
K_unit = np.array([[1, 0, -1, 0], 
                   [0, 0,  0, 0], 
                   [-1, 0, 1, 0], 
                   [0, 0, 0, 0]])

# Generate connectivity matrix, ONLY FOR TRIANGULAR TRUSSES. Order of elements: left element of triangle, bottom horizontal element of triangle, right element of triangle, top horizontal element of triangle to next triangle (only if right support is not reached)
connectivity = np.zeros((nelements+1,2,))
i = 0
j = 0
while i<nnodes-2:
    connectivity[j] = [i,i+1]
    j = j + 1
    connectivity[j] = [i,i+2]
    j = j + 1
    connectivity[j] = [i+1,i+2]
    j = j + 1
    if i < nnodes - 2:
        connectivity[j] = [i+1,i+3]
        j = j + 1
    i = i+2

# Theta generation for each element, for symmetric triangular trusses only
i = 0
while i < nelements:
    theta.append(theta1)
    i += 1
    theta.append(0)
    i += 1
    theta.append(180-theta1)
    i += 1
    if i < nelements:
        theta.append(0)
        i += 1

Pglob = np.array([P*np.cos(np.radians(phiP)), P*np.sin(np.radians(phiP))]) #N P force in global coordinates (x,y)
#Pglob = np.array([-3000000,-18000000]) 

# Rotation matrices T[i], length of elements L[i], stiffness of elements k[i] and stiffness matrix in global coord k_glob[i] generation (i is the element).
# T, L, k and k_glob are arrays containing respectively the rotation matrix, length, stiffness and stiffness matrix of each elements.

T = np.zeros((nelements,4,4))
L = []
k = []
k_glob = np.zeros((nelements,4,4))

for i in range(nelements):
    c = np.cos(np.radians(theta[i]))
    s = np.sin(np.radians(theta[i]))
    if theta[i] == 0:
        Li = 2*a
    if theta[i] == -90:
        Li = b
    else:
        Li = abs(a/c)
    Ai = A[i]
    Ei = E[i]
    ki = Ei * Ai / Li
    T[i] = np.array([[c, s, 0, 0], 
              [-s, c, 0, 0], 
              [0, 0, c, s], 
              [0, 0, -s, c]])
    L.append(Li)
    k.append(ki)

for i in range(nelements):
    k_glob[i] = k[i] * T[i].T @ K_unit @ T[i]


# Assemble global stiffness matrix
NDoF = 2*nnodes
K_global = np.zeros((NDoF,NDoF))
K_globals = np.zeros((nelements,NDoF, NDoF))


# Assemble elements
for i in range (nelements):
    connectelem = connectivity[i] #nodes connected to the element
    indexs = 2*connectelem
    i11 = int(indexs[0])
    i12 = int(indexs[0])+2
    i21 = int(indexs[1])
    i22 = int(indexs[1])+2

    Ki_global = np.zeros((NDoF, NDoF))
    ki_global = k_glob[i]
    Ki_global[i11:i12, i11:i12] = ki_global[0:2, 0:2]
    Ki_global[i21:i22, i11:i12] = ki_global[2:4, 0:2]
    Ki_global[i11:i12, i21:i22] = ki_global[0:2, 2:4]
    Ki_global[i21:i22, i21:i22] = ki_global[2:4, 2:4]
    K_global = K_global + Ki_global
    K_globals[i] = Ki_global


# Calculate displacements at nodes with forces applied
F_red = np.zeros((2*(nnodes-2)))
j = 0
for i in range(nnodes):
    if i in nodes_P:
        F_red[2*j] = Pglob[0]
        F_red[2*j+1] = Pglob[1]
        j += 2
K_red = K_global[2:2*nnodes-2,:]
K_red = K_red[:,2:2*nnodes-2]

u_red = np.linalg.inv(K_red) @ F_red

# Calculate reaction forces at other nodes

u_vec = np.zeros((2*nnodes))

for i in range(nnodes):
    if i != 0 and i != nnodes-1:
        u_vec[2*i] = u_red[2*i-2]
        u_vec[2*i+1] = u_red[2*i+1-2]

F = K_global @ u_vec

axialF = []

# Calculate axial forces
for i in range(nelements):
    F_elei = K_globals[i] @ u_vec
    index_red = 2*connectivity[i]
    i0 = int(index_red[0])
    i1 = int(index_red[1])
    F_elei_red = F_elei[[i0, i0+1, i1, i1+1]]
    F_elei_local = T[i] @ F_elei_red
    Fi = F_elei_local[2]
    axialF.append(Fi)

for i in range(nelements):
    print(f"Axial load in element {i+1}: {round(axialF[i])} N")
    print()

for i in range(nnodes):
    print(f"Node {i+1}, horizontal displacement: {round(u_vec[2*i],2)} mm")
    print(f"Node {i+1}, vertical displacement: {round(u_vec[2*i+1],2)} mm")
    print()
    print(f"Node {i+1}, horizontal force: {round(F[2*i])} N")
    print(f"Node {i+1}, vertical force: {round(F[2*i+1])} N")
    print()
    print()

Axial load in element 1: -281250 N

Axial load in element 2: -37500 N

Axial load in element 3: -93750 N

Axial load in element 4: -225000 N

Axial load in element 5: -93750 N

Axial load in element 6: 75000 N

Axial load in element 7: 93750 N

Axial load in element 8: -225000 N

Axial load in element 9: 93750 N

Axial load in element 10: -37500 N

Axial load in element 11: 281250 N

Node 1, horizontal displacement: 0.0 mm
Node 1, vertical displacement: 0.0 mm

Node 1, horizontal force: 206250 N
Node 1, vertical force: 225000 N


Node 2, horizontal displacement: 0.34 mm
Node 2, vertical displacement: -1.13 mm

Node 2, horizontal force: 0 N
Node 2, vertical force: -150000 N


Node 3, horizontal displacement: -0.06 mm
Node 3, vertical displacement: -1.72 mm

Node 3, horizontal force: 0 N
Node 3, vertical force: 0 N


Node 4, horizontal displacement: 0.0 mm
Node 4, vertical displacement: -2.06 mm

Node 4, horizontal force: 0 N
Node 4, vertical force: -150000 N


Node 5, horizontal displac